## Photometric catalogs - Part 1: Star-Galaxy Discrimination and galaxy number Counts.

C. Stubbs, Feb 14, 2025
Andrés A. Plazas Malagón, Sep. 26, 2025,
With assistance from ChatGPT and Gemini.

### **Overview**

This is the first of a pair of notebooks that provide a database of **photometry, size measurements, and photometric uncertainties** for sources around the galaxy clusters **Abell 1942** and **Abell 360**.  

In this first notebook, we'll use data from the Sloan Digital Sky Survey (SDSS) from a galaxy cluster, Abell 1942, to learn how to separate galaxies from stars, and how to use galaxy counts as a function of brightness to test cosmological predictions.  

In the second notebook, we'll use data from the Vera C. Rubin observatory from another cluster, Abell 360, to select red galaxies that are likely cluster members.  


Here is a link to an image of Abell cluster 1492 in the DESI viewer. :
https://www.legacysurvey.org/viewer#abell%201492

(type A1942 in the search bar and then zoom in a bit).


### **Learning Objectives**

After working through this notebook you will have a good feel for how astronomers work with catalogued datasets that contain parametric descriptors for celestial objects. We start with broadband images of the sky taken in different optical and infrared passbands. Software is used to identify and characterize sources, and merge the detections into assigned attributes of distinct objects.

You'll learn how to discriminate between stars and galaxies, using shape and flux information.

You'll have the chance to fit models to data, and to compare cosmological predictions with observations.

You'll learn how to select out a subset of objects from a parent catalog, and make a color-magnitude diagram for galaxies (!). You'll then play around with selecting a best-size aperture within which to select cluster members, and use their mean color to estimate the redshift of Abell 1942.

### **Submitting Your Work**  

As you work through this, you'll see places where you're asked to answer questions or to enter data. This is done by editing the appropriate text cell.

You'll upload your version of this finished notebook to Canvas,
with the file name

***ClusterLRG_LastFirst_LastFirst.ipynb***

where the LastFirst names are the team members.
Go ahead and save this now, in accord with that naming convention.

### **Steps in This Notebook**
0. Rename and save this notebook in accord with the naming convention described above.
1. Download SDSS photometric data, including positions, magnitudes, and associated uncertainties, for a roughly 1 degree x 1 degree patch of the sky. Transfer the file to your Google Drive.
2. Select out catalog objects that satisfy a cut in signal to noise ratio, SNR.
3. Separate the galaxies from the stars.
4. Make a plot of number counts of high SNR galaxies, and compare to predictions. Woah.
5. Devise selection criteria to pick out the red elliptical galaxies.
6. Estimate the cluster redshift using the color of cluster members.
7. Double-check that you've filled in the notebook with your work.
8. Make sure you filled out the Acknowledgements panel.
9. Save a final version with appropriate naming convention.
10. Submit on Canvas in advance of deadline.
11. Relax and celebrate.

### Tips and Hints

We'll make liberal use of the Gemini AI agent that is embedded in Colab. Make sure you understand the difference between the genric chat bot you invoke with the icon at the top right, vs. the one that interacts more directly with code cells.

In some of the code cells I've left in the prompt that was used to intially create the cell. In many cases it was lightly edited after that initial creation, though.

Remember you can click on the three dots on the right of a code cell and have it "Explain Code" to you.

Places we are seeking your input are flagged like this:

<font color="red"> **your input needed here** </font>

### **Why SDSS?**
*The Sloan Digital SKy Survey (SDSS)* was the first modern wide-field, multi-band imaging plus spectroscopy survey.
Photometric and spectroscopic catalogs are readily available, and it's a good starting point for us.  

Open the Gemini chat window with the icon in the top right of the browser interface. Then cut-and-paste this into the Gemini chat window:



---



*Tell me about the SDSS, the digital sky survey. How many objects did it measure? What was its sky coverage? What telescope did it use? I'm told it used drift scanning for the imaging survey, what is that? I'm also told that optical fibers were used for spectroscopy. How were the fibers positioned at sources of interest?*


---

### **Data File**

You will be using the data file *A1942nohead.tsv*. It needs to be placed on your Google Drive in the same directory where your notebook is stored. You can fetch it from the course Canvas site. First put it on your local computer, then go to your Google Drive folder and use File Upload to send it there.

We are going to use measurements of the apparent brightness of objects in a data file centered on RA=219.59 and DEC=+3.67. Let's do a quick review of astronomical magnitudes. Cut and paste this into the Gemini chat window:

---

*Please provide me with a quick summary of photometric measurements and the definition of astronomical magnutides. Ask me a couple of simple questions to ensure that I understand the flux ratio between objects that differ by 5 magnitudes, and the sense of whether brighter or fainter objects have larger magnutides. Pause to obtain a response to each question before proceeding.*

---


### **Expected Data Structure**
- A structured table with:
  - **RA, Dec** (coordinates)
  - **Object ID**
  - **PSF Magnitudes & Uncertainties** in *u,g,r,i*, and *z* bands
  - **Petrossian magnitudes and Uncertainties (same bands).**

### **Photometry of Point Sources and Extended Sources**

As seen from a ground-based telescope, point sources (such as stars and galaxies that are so small they look like points) are blurred by turbulence in the atmospehre (what astronomers call 'seeing'). These point sources produce two-dimensional Gaussian flux distributions on the focal plane. That 2-d distribution is called the Point Spread Function (PSF). Adding up the flux within that PSF produces a measured quantity called the PSF magnitude, along with its associated uncertainty. The width of the PSF is usually called the Full Width at Half Maximum (FWHM), and typically subtends an angle of order 1 arc second.  

Extended sources, like galaxies, produce smudges of light that are wider than the PSF FWHM. Astronomical codes that measure the brightness of these extended sources use some simple mathematical model that includes a parameterization of shape as well as flux.

The apparent angular size of a galaxy is a rather complicated function of its distance from us. Since we don't know ahead of time how far away a galaxy is,  it's useful to define a measurement of the magnitude of a galaxy that is largely insensitive to the relationship between size and distance. 'Petrosian magnitudes' do this, by adding up the flux within a certain radius away from the center of the object.

Cut and paste this prompt into the Gemini chat box:



---

*Please give me a quick summary of how Petrosian magnitudes work, and why it's a good method for measuring the apparent magnitudes of galaxies. For a star, are the Petrosian and PSF magnitudes the same? What about for resolved galaxies? Also, remind me what a 'resolved' object means*

---


Here's an illustration of the brightness vs. position in x and y on the focal plane of a telescope:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def gaussian_psf(x, y, amplitude, x0, y0, sigma_x, sigma_y):
    """
    Compute a 2D Gaussian Point Spread Function (PSF).

    Parameters
    ----------
    x : array_like
        x-coordinates (can be a grid or 1D array).
    y : array_like
        y-coordinates (same shape as `x`).
    amplitude : float
        Peak value of the Gaussian (maximum flux).
    x0 : float
        Center position in the x-direction.
    y0 : float
        Center position in the y-direction.
    sigma_x : float
        Standard deviation (width) along the x-axis.
    sigma_y : float
        Standard deviation (width) along the y-axis.

    Returns
    -------
    flux : ndarray
        The flux values at each (x, y) position, with the same shape as `x` and `y`.

    Notes
    -----
    This function is commonly used to approximate the brightness distribution
    of a star on a telescope’s focal plane.
    """
    exponent = -((x - x0)**2 / (2 * sigma_x**2) + (y - y0)**2 / (2 * sigma_y**2))
    return amplitude * np.exp(exponent)

# Create a grid of x and y values (focal plane positions)
x = np.linspace(-5, 5, 100)
y = np.linspace(-5, 5, 100)
x, y = np.meshgrid(x, y)

# Define Gaussian PSF parameters
amplitude = 1.0   # Peak flux
x0, y0 = 0.0, 0.0 # Center position
sigma_x = 1.0     # Width in x
sigma_y = 1.0     # Width in y

# Calculate the flux distribution on the focal plane
flux = gaussian_psf(x, y, amplitude, x0, y0, sigma_x, sigma_y)

# Plot the 3D surface of the flux distribution
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(x, y, flux, cmap='viridis')

# Add labels and title
ax.set_xlabel('x (pixels)')
ax.set_ylabel('y (pixels)')
ax.set_zlabel('Flux (arbitrary units)')
ax.set_title('Flux Distribution of a Star (Gaussian PSF)')

plt.show()


In [ ]:
# prompt: I need to open my local Google drive and cd to the Colab notebooks folder
# this is commented out to show a different method, left here as a fallback/mitigation!

#from google.colab import drive
#drive.mount('/content/drive')


#%cd /content/drive/MyDrive/Colab\ Notebooks


In [ ]:
# Here's a slightly different method to upload a data file.
# Download the file A1942_nohead.tsv from Canvas onto your computer, probably into
# your laptop's Downloads directory. Running this cell will allow you to designate that file for
# upload into your Colab Notebooks directory on your remote Google Drive.

# go ahead and run this, click on Choose Files and then find and select A1942_nohead.tsv for transfer.
# the designation .tsv means the columns are separated by TAB characters, so it's a Tab Separated Value (tsv) file.

from google.colab import files
uploaded = files.upload()

In [ ]:
# prompt: I have a tab-delimited data file with the structure listed below. I want to read it into a pandas data frame called FullCatalog_df.
# The name of the input file is A1942_nohead.tsv. Here are the first two lines:
# _r      _RAJ2000        _DEJ2000        mode    cl      SDSS9   RA_ICRS DE_ICRS ObsDate Q       umag    e_umag  gmag    e_gmag  rmag    e_rmag  imag    e_imag  zmag    e_zmag        gpmag   e_gpmag gPmag   e_gPmag gdVrad  gdVell  rpmag   e_rpmag rPmag   e_rPmag rPrad   rdVell  ipmag   e_ipmag iPmag   e_iPmag zpmag   e_zpmag zPmag   e_zPmag
# arcmin  deg     deg                             deg     deg     yr              mag     mag     mag     mag     mag     mag     mag     mag     mag     mag     mag     mag           mag     mag     arcsec  arcsec  mag     mag     mag     mag     arcsec  arcsec  mag     mag     mag     mag     mag     mag     mag     mag
# ------- ------------    ------------    -       -       -------------------     ----------      ----------      ---------       -       ------  ------  ------  ------  ------        ------  ------  ------  ------  ------  ------  ------  ------  ------  ------- ------- ------  ------  ------  ------  ------- ------- ------  ------  ------  ------        ------  ------  ------  ------
# 21.0543 182.91850900    +34.35843100    1       6       J121140.44+342130.3     182.918509      +34.358431      2004.2827       3       22.385   0.281  19.686   0.014  18.412         0.008  17.784   0.007  17.384   0.014  19.777   0.021  19.711   0.022     0.31   0.050 18.492   0.018  18.431   0.011     1.69   0.050 17.897   0.017  17.759   0.009        17.514   0.019  17.363   0.021
# I need the column names to appear in the data frame

import pandas as pd


# Assuming 'A1942nohead.tsv' is in your current working directory
# If not, specify the full path to the file

# this line reads in the contents of the data file, and puts it into a "Pandas" data structure.
# think of it as a large table with one row per object. The columns are listed above.
# the next bit invokes the read.csv function, where csv means Comma Separated Values,
# except we'll stipulate that the separator between fields is a Tab character instead of a comma.
# also, the first line in the file contains the column headers so we tell it that as well.

# the data are read into a data frame called FullCatalog_df.

FullCatalog_df = pd.read_csv('A1942nohead.tsv', sep='\t', header=[0])


Count the number of sources, and print that out.

In [ ]:

FullCount = len(FullCatalog_df)
print(FullCount)

# take a look at the first few rows of the data structure

FullCatalog_df.head()


In astronomy, we often describe how bright an object is using magnitudes rather than raw fluxes.

Let’s walk through how flux relates to magnitude, what the zero point means, and how to correctly compute errors on magnitudes from flux errors.

### Magnitude–Flux Relationship

The **apparent magnitude** $m$ of a source is related to its measured flux $F$ (in some band, here the *i*-band) through the formula:


$m = -2.5 \, \log_{10}(F) + C$


- $F$: the observed flux of the object.  
- $C$: the **zero point**, a constant that sets the magnitude scale.  

---

### What is the Zero Point?

The **zero point** ensures that the magnitude system matches a standard reference.  
- Historically, the zero point was chosen so that bright stars like Vega have magnitude close to 0 in optical bands.  
- In modern CCD-based surveys, the zero point comes from calibrations that convert instrumental flux counts into physical magnitudes.  

---

### Error Propagation: From Flux to Magnitude

We also need the **uncertainty on the magnitude** $\sigma_m$, given the flux error $\sigma_F$.

The general rule is:

$
\sigma_m = \left| \frac{dm}{dF} \right| \, \sigma_F
$

#### Step 1. Differentiate the magnitude formula:

$
m(F) = -2.5 \log_{10}(F) + C
$

$
\frac{dm}{dF} = -\frac{2.5}{\ln(10)} \cdot \frac{1}{F}
$

#### Step 2. Apply error propagation:

$
\sigma_m = \frac{2.5}{\ln(10)} \cdot \frac{\sigma_F}{F}
$

Note that that $\frac{2.5}{\ln(10)} ≈ 1.0857$, which is pretty darn close to 1.0

This shows that:
- The **relative error in flux** ($\sigma_F/F$) is what matters.  
- For modest variations in flux, the magnitude difference is approximately equal to the fractional flux uncertainity. If sigma_m is 0.1, then fractional uncertainty is 10% and reciprocal of that, the signal to noise ratio, is 10.
- Magnitude errors get larger for faint sources (small $F$) because dividing by $F$ amplifies uncertainties.  


**For reasonable SNR values, the magniture uncertainty is approximately the inverse of the SNR.**


Let's now add another plot, that looks at signal-to-noise ratio for galaxy photometry. For modest values of uncertainty, the uncertainty in magnitudes is roughly the fractional uncertainty in flux (see explanation above). So $\sigma_m$ of 0.1 mag corresponds to a 10% flux uncertainty, or a signal to noise ratio of 10.

Now let’s only include objects that have a decent signal-to-noise ratio.
The uncertainty in a magnitude measurement is (roughly) the fractional uncertainty in flux.
We can start by making a histogram of the galaxy photometry uncertainties in the r-band cModel magnitude, `r_cModelMagErr`.

In [ ]:
import matplotlib.pyplot as plt

# Create figure with two subplots side by side
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True, sharey=False)

# Linear scale
axes[0].hist(FullCatalog_df[('e_rPmag')], bins=3000, edgecolor='black')
axes[0].set_xlabel(r'$r_{\rm cModel}$ uncertainty, $\sigma_r$ (mag)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Histogram (Linear scale)')
axes[0].set_xlim(0, 2)

# Log scale
axes[1].hist(FullCatalog_df[('e_rPmag')], bins=3000, edgecolor='black')
axes[1].set_xlabel(r'$r_{\rm cModel}$ uncertainty, $\sigma_r$ (mag)')
axes[1].set_ylabel('Frequency (log scale)')
axes[1].set_title('Histogram (Log scale)')
axes[1].set_xlim(0, 2)
axes[1].set_yscale('log')

plt.tight_layout()
plt.show()





In [ ]:
# Let's look at the median uncertainty
print( np.median(FullCatalog_df['e_rPmag']))

So if we impose a cut for objects with SNR>=5, that correponds to $\sigma_r$ <=0.2. It seems we'll lose about half the objects in the catalog. Let's make a subset of highSNR objects by imposing this cut.


In [ ]:
sigma_mag_cut = 0.2

# Apply SNR cut: keep objects with r-band cModel magnitude error <= sigma_mag_cut mag
HighSNR_df = FullCatalog_df[FullCatalog_df['e_rPmag'] <= sigma_mag_cut]

# Compare the number of rows
original_count = len(FullCatalog_df)
highsnr_count = len(HighSNR_df)

print(f"Original DataFrame has {original_count} rows.")
print(f"HighSNR DataFrame has {highsnr_count} rows.")


## Star-galaxy separation

Our next task is to separate stars from galaxies. There are a couple of ways to do this. One is to compare PSF magnitudes, which does photometry assuming a 2d Gaussian point source, to Petrosian magnitudes that allow for a broader shape. A standard metric is to compute (PSF magnitude - Petrosian magnitude), and for galaxies we'll exclude cases where that is close to zero. A standard metric is to compute (`drmag` = PSF magnitude – Petrossian magnitude). Stars have values close to zero, while galaxies deviate from zero because of their extended shapes.

In [ ]:
import matplotlib.pyplot as plt

# Make sure drmag is defined with the new quantities
HighSNR_df = HighSNR_df.copy()
HighSNR_df['drmag'] = HighSNR_df['rpmag'] - HighSNR_df['rPmag']

# Create figure with two subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 6), sharex=True)

# Zoomed-in view (typical star/galaxy separation)
axes[0].plot(HighSNR_df['rPmag'], HighSNR_df['drmag'], 'o', markersize=2)
axes[0].set_xlabel('rPmag')
axes[0].set_ylabel('drmag = rpmag – rPmag')
axes[0].set_title('Zoomed-in: Star/Galaxy Separation')
axes[0].set_ylim(-0.2, 1)

# Full range view (to catch outliers)
axes[1].plot(HighSNR_df['rPmag'], HighSNR_df['drmag'], 'o', markersize=2)
axes[1].set_xlabel('rPmag')
axes[1].set_title('Larger drmag range')
# axes[1].set_ylim(-1, 8)

plt.tight_layout()
plt.show()



**In any selection there is a tradoff between purity and completeness.**

Let's allow a few stars to leak in with a fairly loose cut, but always keep this tradeoff in mind.

We can use this distribution to discriminate between stars and galaxies. The PSFMag values assume a Gaussian point source, while the Petrossian magnitude values are more flexible and add up flux in a larger aperture. If these two are the same, it's likely to be a star.

Sorry about the lousy notation (blame SDSS!) but remember that rpmag is PSF magnitude and rPmag is a Petrossian magnitude (name for Vahe Petrossian of Stanford).

Let's define a `star_galaxy_cut` of `0.1` in `drmag` to separate the stars and the galaxies, and the visualize with different colors and symbols de stellar locus.



In [ ]:
import numpy as np


star_galaxy_cut = 0.1

# Select galaxies: those with |drmag| > star_galaxy_cut

#HighSNRGalaxies = HighSNR_df[np.abs(HighSNR_df['drmag']) > star_galaxy_cut].copy()

stars = HighSNR_df[HighSNR_df['drmag'].abs() < star_galaxy_cut]
galaxies = HighSNR_df[HighSNR_df['drmag'].abs() >= star_galaxy_cut]

print(f"HighSNR sample: {len(HighSNR_df)} objects")
print(f"High SNR Galaxies: {len(galaxies)} objects")
galaxies.head()



In [ ]:
import matplotlib.pyplot as plt

# Create the plot
plt.figure(figsize=(10, 6))

# Plot stars
plt.plot(stars['rPmag'], stars['drmag'],
         'o', color='blue', markersize=2, label=f'Stars (|drmag| < {star_galaxy_cut})')

# Plot galaxies
plt.plot(galaxies['rPmag'], galaxies['drmag'],
         's', color='red', markersize=3, label=f'Galaxies (|drmag| ≥ {star_galaxy_cut})')

# Add horizontal dashed lines for the separation threshold
# plt.axhline(0.02, color='gray', linestyle='--', linewidth=1)
# plt.axhline(-0.02, color='gray', linestyle='--', linewidth=1)

# Labels and title
plt.xlabel('rPmag')
plt.ylabel('drmag = rpmag – rPmag')
plt.title('Star/Galaxy Separation in High SNR sample')

# Legend
plt.legend()

plt.show()


## Completeness Determination
There are more faint sources than bright ones, for a variety of reasons. This means we should expect a histogram of the number of sources vs. their magnitude to continuously increase. Let's make a plot of the number of sources vs. magnitude, as well as a cumulative plot that takes the integral of number of sources down to some faintness limit $f$, as a function of $f$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Histogram of rPmag magnitudes for galaxies
plt.figure(figsize=(10, 6))
plt.hist(galaxies['rPmag'], bins=40, edgecolor='black')
plt.xlabel(r'$r_{P}$ (mag)')
plt.ylabel('Frequency')
plt.title('Histogram of rPmag Magnitudes (Galaxies)')
plt.show()

# Cumulative counts plot
plt.figure(figsize=(10, 6))
values, base = np.histogram(galaxies['rPmag'], bins=40)

cumulative = np.cumsum(values)
plt.plot(base[:-1], cumulative, 'ko-')
plt.xlabel(r'$r_{P}$ (mag)')
plt.ylabel('Cumulative Count')
plt.title('Cumulative Count Plot of rPmag Magnitudes (Galaxies)')
plt.yscale('log')
plt.show()






Take a look at the two plots produced above. Is there a threshold magnitude fainter than which there aren't many objects in the catalog of high-SNR galaxies? Remember that larger numbers correspond to fainter objects!  Why might this happen?

<font color="red"> **your input needed here** </font>

The magnitude where the cumulative counts flatten out is the 'magnitude limit' of the catalog.

It's important to remember that the completness of the catalog depends on the distance to the kind of object in question, and that we're plotting *apparent* magnitudes, not *absolute* magnitudes. What's the difference?

<font color="red"> **your input needed here** </font>



### Do we live in a static and Euclidean universe?

Imagine the Universe was static and Euclidean, and that there was only one kind of galaxy with a single brightness. The number of objects within a sphere of radius R grows as R$^3$. But the ones farther away are fainter. If this is true for one single kind of galaxy, it's also true for a mixture of types.

Cosmologists like to produce plots of 'Number Counts', which is the number of observed objects brighter than some magnitude $m$ vs. $m$.
We usually show this with a logarithmic y axis.

Cut and paste this prompt into the Gemini AI chat window:


---

**Prompt**: *Give me an explanation for the predicted value of the slope of the number counts of galaxies as a function of magnitude, for a static Euclidean Universe.*

---

What's the predicted slope for a static Euclidean universe?

<font color="red"> **your input needed here** </font>


We can compare our observed number count slope with this prediction.
The cell below makes a fit to the number count plot for the high-SNR galaxies we selected.

Remember you can always select the cell and then choose Explain Code from the menu under the three dots on the right of the icon panel!



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

def power_law(m, a, b):
    """
    Power-law model for galaxy number counts in log-log space.

    Parameters
    ----------
    m : array_like
        Magnitudes (e.g., rPmag).
    a : float
        Intercept term (log10 normalization).
    b : float
        Slope of the relation (log10 counts vs. magnitude).

    Returns
    -------
    log_counts : ndarray
        Predicted log10 of cumulative counts for each magnitude.

    Notes
    -----
    This model assumes:
        log10(N) = a + b * m
    where N is the cumulative number of galaxies brighter than magnitude m.
    """
    return a + b * m

# ---------------------------------------------------------------------
# Step 1: Set magnitude range for fitting
# ---------------------------------------------------------------------
mag_min = 15.0      # lower limit of magnitudes to include in the fit
mag_max = 20.5      # upper limit of magnitudes to include in the fit

# ---------------------------------------------------------------------
# Step 2: Build cumulative counts of galaxy magnitudes
# ---------------------------------------------------------------------
plt.figure(figsize=(10, 6))

# Histogram of magnitudes (30 bins by default)
values, base = np.histogram(galaxies['rPmag'], bins=30)

# Compute cumulative counts (number of galaxies brighter than each bin edge)
cumulative = np.cumsum(values)

# ---------------------------------------------------------------------
# Step 3: Select data within the chosen magnitude range for fitting
# ---------------------------------------------------------------------
mask = (base[:-1] >= mag_min) & (base[:-1] <= mag_max)
mag_fit = base[:-1][mask]
counts_fit = cumulative[mask]

# Avoid log10(0) by keeping only strictly positive counts
counts_fit = counts_fit[counts_fit > 0]
mag_fit = mag_fit[:len(counts_fit)]  # Ensure arrays have same length

# ---------------------------------------------------------------------
# Step 4: Perform log-log fit
# ---------------------------------------------------------------------
log_counts_fit = np.log10(counts_fit)               # take log10 of counts
params, covariance = curve_fit(power_law, mag_fit, log_counts_fit)
a_fit, b_fit = params
slope_uncertainty = np.sqrt(np.diag(covariance))[1]  # uncertainty in slope b

# ---------------------------------------------------------------------
# Step 5: Plot data and fit
# ---------------------------------------------------------------------
# Plot cumulative data
plt.plot(base[:-1], cumulative, 'co-', label="Data")
plt.yscale('log')  # log scale highlights power-law behavior

plt.xlabel(r'$r_{P}$ (mag)')
plt.ylabel('Cumulative Count')
plt.title('Cumulative Count Plot of rPmag Magnitudes')

# Plot fitted model
mag_range = np.linspace(mag_min, mag_max, 100)
plt.plot(mag_range, 10**power_law(mag_range, *params),
         'r--', label=f'Fit: slope = {b_fit:.3f} ± {slope_uncertainty:.3f}')

plt.legend()
plt.show()

# ---------------------------------------------------------------------
# Step 6: Print slope (b) of the fitted relation
# ---------------------------------------------------------------------
print(f"Fitted slope: {b_fit:.3f} ± {slope_uncertainty:.3f}")



## A Comparison: Confronting a Prediction with Data ##
OK, we have a measurement of the number count slope, and we have a prediction. How do they compare?? Is there a statistically significant difference between the prediction and the measurement? By how many standard deviations do they differ? The cell below will help make that calculation. If we have two quantities $A$ and $B$ with associated uncertainties $\sigma_A$ and $\sigma_B$, their difference has an uncertainty of $\sigma_\Delta=\sqrt{\sigma_A^2 + \sigma_B^2}$. The statistical significance of the difference, in units of standard deviations, is then |A-B|/$\sigma_\Delta$.

How do we interpret this? Well... in the absence of better information we make an assumption of Gaussian statistics, so that roughly 68% of the time we should get less than one sigma of difference. A three sigma discrepancy is only supposed to happen by chance less than 1% of the time, and 5 standard deviations is considered iron-clad evidence for a discrepancy.

Do take note, however, that something like 2 million technical papers are published per year.

**That means some unlucky scientist somewhere is gloating or fretting about a one-in-a-million statistical result, but they're just the victom of bad luck.**

Here's a wallet card version:

| Number of Sigmas | How Frequent? | What to Bet  |
|---|---|---|
| 1 | 32% | A Coffee |
| 2 | 4.5% |A Lunch |
| 3 | 0.27% | A (cheap) Car |
| 5 | 0.000057% | A House |
| 10| 1.52 x 10$^{-23}$% |Your Entire Wealth |

I have an entire rant about why this is all misleading, however. Here is the terse version:

1. The world isn't Gaussian. That's a just a model, and it's wrong.
2. Most experiments underestimate their uncertainties. A factor of two underestimate has an **exponential** impact on implied significance. So beware the likeilihoods shown above.
3. Especially in cosmology, measurments suffer from sources of systematic (rather than statsitical) errors and those are certainly NOT Gaussian, and are very hard to quantify.


In [ ]:
# Compute significance of number count slope difference. The prediction for a static Euclidean Universe is 0.6
# We will take that as a perfect value, with no uncertainty. So the standard deviation of the difference
# is just the measurment uncertainty that was produced in the fit above.


Nsigma=np.abs(b_fit-0.6)/slope_uncertainty
print(f"Slope discrepancy significance is {Nsigma:.1f} sigma")

## Discussion ##

Is there a significant discrepancy here, or not?

Let's pause and consider what this means. Think of some reasons that could give rise to this result. Is the model that made the prediction ruled out? What might affect the measurements? Think hard about what we're measuring and what inplicit assumptions are being made.

Type in your responses here:

<font color="red"> **your input needed here** </font>




### Acknowledgements

List the resources you used for this exercise, including any use of AI tools.

<font color="red"> **your input needed here** </font>

### **References** ###


Star-galaxy separation:
https://outerspace.stsci.edu/display/PANSTARRS/How+to+separate+stars+and+galaxies



In [ ]:
# this is supposed to generate a PDF file of this on Google Drive.
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('ClusterLRG-Rubin-Part1-2025SEP25.ipynb')